# Training

In [3]:
import cv2
import numpy as np
import time 
import os, glob
from PIL import Image
from skimage import morphology
import matplotlib.pyplot as plt 
import random
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

In [4]:
#base_path = 'save-10-b'
#base_path = 'save-5-g'
base_path = 'save'

In [5]:
train_datagen = ImageDataGenerator(
    validation_split=0.1,
    preprocessing_function=preprocess_input
)

val_datagen = ImageDataGenerator(
    validation_split=0.1,
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_directory(
    base_path + '/train',
    target_size=(300, 300),
    classes=['hela', 'huh-7','MCF-7','NCI'],
    class_mode='categorical',
    batch_size=5,
    shuffle=True,
    subset='training'
)

val_gen = val_datagen.flow_from_directory(
    base_path + '/val',
    target_size=(300, 300),
    classes=['hela', 'huh-7','MCF-7','NCI'],
    class_mode='categorical',
    batch_size=5,
    shuffle=True,
    subset='validation'
)



print(val_gen.class_indices)

Found 49126 images belonging to 4 classes.
Found 1330 images belonging to 4 classes.
{'hela': 0, 'huh-7': 1, 'MCF-7': 2, 'NCI': 3}


In [6]:
for data_batch, labels_batch in train_gen:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break

배치 데이터 크기: (5, 300, 300, 3)
배치 레이블 크기: (5, 4)


In [ ]:
for item in train_gen:
    print(item)
    a = np.array(item[0])
    break

(array([[[[-0.99215686, -0.99215686, -0.99215686],
         [-0.99215686, -0.99215686, -0.99215686],
         [-0.99215686, -0.99215686, -0.99215686],
         ...,
         [-1.        , -1.        , -1.        ],
         [-1.        , -1.        , -1.        ],
         [-1.        , -1.        , -1.        ]],

        [[-0.99215686, -0.99215686, -0.99215686],
         [-0.99215686, -0.99215686, -0.99215686],
         [-0.99215686, -0.99215686, -0.99215686],
         ...,
         [-1.        , -1.        , -1.        ],
         [-1.        , -1.        , -1.        ],
         [-1.        , -1.        , -1.        ]],

        [[-0.99215686, -0.99215686, -0.99215686],
         [-0.99215686, -0.99215686, -0.99215686],
         [-0.99215686, -0.99215686, -0.99215686],
         ...,
         [-1.        , -1.        , -1.        ],
         [-1.        , -1.        , -1.        ],
         [-1.        , -1.        , -1.        ]],

        ...,

        [[-1.        , -1.        , -

In [7]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint

base_model = MobileNetV2(input_shape=(300,300,3), weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

model.summary()

9420800/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 150, 150, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 150, 150, 32  128         ['Conv1[0][0]']                  
                               

In [8]:
for layer in model.layers:
    layer.trainable = True

In [9]:
MODEL_SAVE_FOLDER_PATH = 'hdf5_save/' 

model_path = MODEL_SAVE_FOLDER_PATH + "cancer - " +'{epoch:02d}-{val_loss:.4f}.hdf5'


In [10]:

checkpoint = ModelCheckpoint(model_path,             # file명을 지정합니다
                             monitor='val_acc',   # val_loss 값이 개선되었을때 호출됩니다
                             verbose=1,            # 로그를 출력합니다
                             save_best_only=True,  # 가장 best 값만 저장합니다
                             mode='auto'           # auto는 알아서 best를 찾습니다. min/max)
                            )

In [ ]:
from keras.callbacks import ModelCheckpoint
history = model.fit_generator(train_gen, validation_data=val_gen, epochs=5,callbacks=[checkpoint])

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


1789/1789 [==============================] - ETA: 0s - loss: 0.9737 - acc: 0.5754
Epoch 1: val_acc improved from -inf to 0.36794, saving model to hdf5_save/cancer - 01-6.3549.hdf5
1789/1789 [==============================] - 4241s 2s/step - loss: 0.9737 - acc: 0.5754 - val_loss: 6.3549 - val_acc: 0.3679
Epoch 2/5
1789/1789 [==============================] - ETA: 0s - loss: 0.8340 - acc: 0.6460
Epoch 2: val_acc improved from 0.36794 to 0.38306, saving model to hdf5_save/cancer - 02-3.3844.hdf5
1789/1789 [==============================] - 144s 81ms/step - loss: 0.8340 - acc: 0.6460 - val_loss: 3.3844 - val_acc: 0.3831
Epoch 3/5
1789/1789 [==============================] - ETA: 0s - loss: 0.7712 - acc: 0.6864
Epoch 3: val_acc did not improve from 0.38306
1789/1789 [==============================] - 144s 80ms/step - loss: 0.7712 - acc: 0.6864 - val_loss: 3.4005 - val_acc: 0.3821
Epoch 4/5
1789/1789 [==============================] - ETA: 0s - loss: 0.7091 - acc: 0.7161
Epoch 4: val_acc imp

In [ ]:
model.save('hdf5_save/cancer_cell_model_epochs_10.h5')

In [ ]:
history = model.fit_generator(train_gen, validation_data=val_gen, epochs=10,callbacks=[checkpoint])

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


691/691 [==============================] - ETA: 0s - loss: 0.5326 - acc: 0.7870
Epoch 1: val_acc improved from -inf to 0.31395, saving model to hdf5_save-5-g/cancer - 01-5.1402.hdf5
691/691 [==============================] - 940s 1s/step - loss: 0.5326 - acc: 0.7870 - val_loss: 5.1402 - val_acc: 0.3140
Epoch 2/10
691/691 [==============================] - ETA: 0s - loss: 0.3818 - acc: 0.8542
Epoch 2: val_acc improved from 0.31395 to 0.38372, saving model to hdf5_save-5-g/cancer - 02-2.7818.hdf5
691/691 [==============================] - 55s 79ms/step - loss: 0.3818 - acc: 0.8542 - val_loss: 2.7818 - val_acc: 0.3837
Epoch 3/10
691/691 [==============================] - ETA: 0s - loss: 0.2896 - acc: 0.8858
Epoch 3: val_acc improved from 0.38372 to 0.43023, saving model to hdf5_save-5-g/cancer - 03-5.3980.hdf5
691/691 [==============================] - 54s 79ms/step - loss: 0.2896 - acc: 0.8858 - val_loss: 5.3980 - val_acc: 0.4302
Epoch 4/10
691/691 [==============================] - ETA:

In [ ]:
model.save('hdf5_save-5-g/cancer_cell_model_epochs_10.h5')